# Init

In [1]:
import os, sys

import numpy as np
import matplotlib.pyplot as plt

config = "bruz"
if config == "ur1":
    sys.path.insert(0, "../")
    siepelDir = "C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2"
elif config == "bruz":
    sys.path.insert(0, "/home/pleroy/DEV/DIADEM")
    siepelDir = "/home/pleroy/DATA/CAMILL/WR15_v2"

import toolsdiadem.tools as dia
import toolsdiadem.ser as ser
from toolsdiadem.Probe import Probe
from toolsdiadem.ConfigSer import ConfigSer

In [2]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2

In [3]:
cmap='jet'
configSupport = os.path.join(siepelDir, "wr15_plaqueSupport.ini")
configRef = os.path.join(siepelDir, "wr15_plaqueRef.ini")
confSupport = ConfigSer(configSupport)
confRef = ConfigSer(configRef)
confSupport.printConfig()
confRef.printConfig()

/home/pleroy/DATA/CAMILL/WR15_v2/wr15_plaqueSupport.ini
nb_elev 41
nb_freq 2001
nb_ssb 3
el0 0.7
az0 -0.44
base_path /home/pleroy/DATA/CAMILL/WR15_v2
out_path /home/pleroy/DATA/CAMILL/WR15_v2/processing/wr15
gateWidth 500.0
gateCenter 6450.0
/home/pleroy/DATA/CAMILL/WR15_v2/wr15_plaqueRef.ini
nb_elev 41
nb_freq 2001
nb_ssb 3
el0 0.31
az0 -0.44
base_path /home/pleroy/DATA/CAMILL/WR15_v2
out_path /home/pleroy/DATA/CAMILL/WR15_v2/processing/wr15
gateWidth 0.0
gateCenter 0.0


# Données de la sonde

In [6]:
probe = Probe(os.path.join(confSupport.base_path, "S11FDC_WR15.s1p"))
probe_2020_08_19 = Probe(os.path.join(confSupport.base_path, "S11_PROBE_WR15_2020_08_19.s1p"))
probe_SUPPORT_AU_MAX_PLUS_ABS = Probe(os.path.join(confSupport.base_path, "S11_SUPPORT_AU_MAX_PLUS_ABS_2020_08_19.s1p"))
probe_SUPPORT_AU_MAX_SANS_ABS = Probe(os.path.join(confSupport.base_path, "S11_SUPPORT_AU_MAX_SANS_ABS_2020_08_19.s1p"))
probe_SUPPORT_A_ZERO_SANS_ABS = Probe(os.path.join(confSupport.base_path, "S11_SUPPORT_A_ZERO_SANS_ABS_2020_08_19.s1p"))

In [9]:
fig, ax = plt.subplots(1,1)
ax.plot(probe.d, ser.dB(np.fft.ifft(probe.S11)), label=f"Probe" )
ax.plot(probe.d, ser.dB(np.fft.ifft(probe_2020_08_19.S11)), label=f"SUPPORT_A_ZERO_PLUS_ABS, 2020_08_19" )
ax.plot(probe.d, ser.dB(np.fft.ifft(probe_SUPPORT_AU_MAX_PLUS_ABS.S11)), label=f"SUPPORT_AU_MAX_PLUS_ABS, 2020_08_19" )
ax.plot(probe.d, ser.dB(np.fft.ifft(probe_SUPPORT_AU_MAX_SANS_ABS.S11)), label=f"SUPPORT_AU_MAX_SANS_ABS, 2020_08_19" )
ax.plot(probe.d, ser.dB(np.fft.ifft(probe_SUPPORT_A_ZERO_SANS_ABS.S11)), label=f"SUPPORT_A_ZERO_SANS_ABS, 2020_08_19" )
ax.legend()
ax.grid()

# Données des plaques

## Référence

In [ ]:
dir_ref = f"{confRef.base_path}/20200706/026_plaqueRef"
freq_reference, S11_reference = ser.getData_ssb(confRef.nb_elev, confRef.nb_freq, confRef.nb_ssb, dir_ref)
S11_reference_compensated = S11_reference - probe.S11

In [ ]:
d = np.linspace(0, 300 / (freq_reference[1]-freq_reference[0]), confRef.nb_freq) * 0.5;

In [ ]:
fig, (ax0, ax1) = plt.subplots(1,2)

im = ax0.pcolormesh(confRef.elevation, freq_reference, ser.dB(S11_reference).T, cmap=cmap )
title = f"S11_reference"
ax0.set_title(title)
ax0.set_xlabel( "Elévation [°]" )
ax0.set_ylabel( "Fréquence [GHz]" )
dia.addColorBar(  im, ax0, aspect='auto' )

im = ax1.pcolormesh(confRef.elevation, freq_reference, ser.dB(S11_reference_compensated).T, cmap=cmap )
title = f"S11_reference_compensated"
ax1.set_title(title)
ax1.set_xlabel( "Elévation [°]" )
ax1.set_ylabel( "Fréquence [GHz]" )
dia.addColorBar(  im, ax1, aspect='auto' )

## Support

In [ ]:
data_dir = f"{confSupport.base_path}/20200701/plaqueSupport"
freq_support, S11_support = ser.getData_ssb(
    confSupport.nb_elev, confSupport.nb_freq, confSupport.nb_ssb, data_dir)
S11_support_compensated = S11_support - probe.S11

In [ ]:
fig, (ax0, ax1) = plt.subplots(1,2)

im = ax0.pcolormesh(confSupport.elevation, freq_support, ser.dB(S11_support).T, cmap=cmap )
title = f"S11_support"
ax0.set_title(title)
ax0.set_xlabel( "Elévation [°]" )
ax0.set_ylabel( "Fréquence [GHz]" )
dia.addColorBar(  im, ax0, aspect='auto' )

im = ax1.pcolormesh(confSupport.elevation, freq_support, ser.dB(S11_support_compensated).T, cmap=cmap )
title = f"S11_support_compensated"
ax1.set_title(title)
ax1.set_xlabel( "Elévation [°]" )
ax1.set_ylabel( "Fréquence [GHz]" )
dia.addColorBar(  im, ax1, aspect='auto' )

# Recherche de maximum

In [ ]:
idx = np.where(abs(d - confSupport.gateCenter) < confSupport.gateWidth)[0]

## Référence

In [ ]:
maxVals = np.amax(np.abs(np.fft.ifft(S11_reference, axis=1))[:,idx], axis=1)
maxElev = np.where(maxVals == np.amax(maxVals))
print(f"maxElev = {maxElev}")
iElevRef = maxElev[0][0]
print(f"[b] elevationPlaque at maximum: {confRef.elevation[iElevRef]:.2f}")

## Support

In [ ]:
maxVals = np.amax(np.abs(np.fft.ifft(S11_support, axis=1))[:,idx], axis=1)
maxElev = np.where(maxVals == np.amax(maxVals))
print(f"maxElev = {maxElev}")
iElevSupportMax = maxElev[0][0]
print(f"[b] elevationSupport at maximum: {confSupport.elevation[iElevSupportMax]:.2f}")

## Plot

In [ ]:
ref_max = S11_reference[iElevRef,:]
support_max = S11_support[iElevSupportMax,:]
refComp_max = S11_reference_compensated[iElevRef,:]
supportComp_max = S11_support_compensated[iElevSupportMax,:]

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(d, ser.dB(np.fft.ifft(ref_max)), label=f"ref_max, {confRef.elevation[iElevRef]:.2f}")
ax.plot(d, ser.dB(np.fft.ifft(refComp_max)), label=f"refComp_max, {confRef.elevation[iElevRef]:.2f}")
ax.plot(d, ser.dB(np.fft.ifft(support_max)), label=f"Support_max, {confSupport.elevation[iElevSupportMax]}")
ax.plot(d, ser.dB(np.fft.ifft(supportComp_max)), label=f"SupportComp_max, {confSupport.elevation[iElevSupportMax]}")
ax.legend()
ax.grid()

# Save data

In [ ]:
tmp = np.zeros((confRef.nb_freq, 3))
tmp[:,0] = freq_reference
tmp[:,1] = np.real(ref_max)
tmp[:,2] = np.imag(ref_max)
file_ref = f"{confRef.out_path}/data/WR15_S11_plaque_ref.txt"
np.savetxt(file_ref, tmp, header="GHz real imag")
print(file_ref)

In [ ]:
tmp = np.zeros((confRef.nb_freq, 3))
tmp[:,0] = freq_reference
tmp[:,1] = np.real(support_max)
tmp[:,2] = np.imag(support_max)
file_support = f"{confSupport.out_path}/data/WR15_S11_plaque_support.txt"
np.savetxt(file_support, tmp, header="GHz real imag")
print(file_support)